In [0]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=32964c800abcec0d982d6bec25558bbef499444416f5de0f32b248fd50932be0
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS 508/Assignment 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS 508/Assignment 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...    

In [0]:
X_train = train_data.iloc[:, :-1]
y_train = train_data["QuoteConversion_Flag"]
X_test = test_data.iloc[:, :-1]
test_data["QuoteNumber"]


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)

In [39]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=3, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


{'min_samples_split': 50, 'max_depth': 9}
=== All AUC Scores ===
[0.93883245 0.94598483 0.91696121]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9339261639826476


In [0]:
final_predict=pd.DataFrame(clf_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/dt_result.csv", index = None)

In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)


In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)

#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


In [0]:
final_predict1=pd.DataFrame(rfc_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict1],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/rf_result.csv", index = None)

In [0]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)

#Randomized Search for hyperparameter tuning
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)

In [0]:
#Construct Gradient Boosting Trees using the best parameters
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,y_train)
abc_predict = abc.predict(X_test)

#run cross-validation on best parameters, get auc score
abc_cv_score = cross_val_score(abc, X_train, y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())


In [0]:
final_predict2=pd.DataFrame(abc_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict2],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/GB_result.csv", index = None)

In [0]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
mlp_predict = mlp.predict(X_test)
mlp_cv_score = cross_val_score(mlp, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",mlp_cv_score.mean())

In [0]:
final_predict3=pd.DataFrame(mlp_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict3],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/MLP_result.csv", index = None)

In [34]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predict = knn.predict(X_test)
#Hyperparameter tuning for knn
parameters={'n_neighbors' : [3,4,5], 'leaf_size': [20,30,40,50]}
knn_random = RandomizedSearchCV(knn,parameters,n_iter=15)
knn_random.fit(X_train, y_train)
grid_parm_knn=knn_random.best_params_
print(grid_parm_knn)
#Using the parameters obtained from HyperParameterTuning in the KNeighborsClassifier
knn = KNeighborsClassifier(**grid_parm_knn)
knn.fit(X_train,y_train)
knn_predict = knn.predict(X_test)
#run cross-validation on best hyperparameters, get auc score
knn_cv_score = cross_val_score(knn, X_train, y_train, cv=3, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ",knn_cv_score.mean())


{'n_neighbors': 4, 'leaf_size': 20}
=== All AUC Scores ===
[0.50931049 0.50636385 0.5089168 ]


=== Mean AUC Score ===
Mean AUC Score - KNN:  0.5081970473337238


In [0]:
final_predict4=pd.DataFrame(knn_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict4],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/knnnew_result.csv", index = None)

In [37]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)
lsvc_predict = lsvc.predict(X_test)

#Hyperparameter Tuning for LinearSVC
parameters={'max_iter' : [10,20],'loss': ['hinge','squared_hinge']}
lsvc_random = RandomizedSearchCV(lsvc,parameters,n_iter=15)
lsvc_random.fit(X_train, y_train)
grid_parm_lsvc=lsvc_random.best_params_
print(grid_parm_lsvc)

#Using the parameters obtained from HyperParameterTuning in the LinearSVC
lsvc = LinearSVC(**grid_parm_lsvc)
lsvc.fit(X_train,y_train)
lsvc_predict = lsvc.predict(X_test)

#run cross-validation on best hyperparameters, get auc score
lsvc_cv_score = cross_val_score(lsvc, X_train, y_train, cv=3, scoring="roc_auc")
print("=== All AUC Scores ===")
print(lsvc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Linear SVC: ",lsvc_cv_score.mean())

{'max_iter': 10, 'loss': 'squared_hinge'}
=== All AUC Scores ===
[0.58324578 0.66281529 0.65045404]


=== Mean AUC Score ===
Mean AUC Score - Linear SVC:  0.6321717048900967


In [0]:
final_predict5=pd.DataFrame(lsvc_predict,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],final_predict5],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/lsvcnew_result.csv", index = None)

In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93735464]
    fold  1:  [0.93649188]
    fold  2:  [0.93567961]
    fold  3:  [0.93087581]
    ----
    MEAN:     [0.93510049] + [0.00251000]
    FULL:     [0.93510056]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91541106]
    fold  1:  [0.91282803]
    fold  2:  [0.90989078]
    fold  3:  [0.91029531]
    ----
    MEAN:     [0.91210629] + [0.00221532]
    FULL:     [0.91210639]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90196178]
    fold  1:  [0.90332204]
    fold  2:  [0.90200243]
    fold  3:  [0.89755259]
    ----
    MEAN:     [0.90120971] + [0.00218120]
    FULL:     [0.90120975]



In [40]:
#STACKING - CONTRUCT A Decision Tree MODEL==============================
model = DecisionTreeClassifier()
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,100,5),'max_depth': range(5,30,3),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(model,parameters,n_iter=15,cv=5)
clf_random.fit(X_train, y_train)
grid_parm_sdt=clf_random.best_params_
print(grid_parm_sdt)

clf = DecisionTreeClassifier(**grid_parm_sdt)
    
model = clf.fit(S_Train, y_res)
y_pred = clf.predict(S_Test)
stacked_predict1=pd.DataFrame(y_pred,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],stacked_predict1],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/stackedDT_result.csv", index = None)


RandomizedSearchCV-Decision tree
{'min_samples_leaf': 15, 'max_depth': 11, 'criterion': 'gini'}


In [42]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model2 = RandomForestClassifier()
print("RandomizedSearchCV-Random Forest")
parameters={ 'n_estimators': range(50,150,30),'min_samples_split' : range(50,100,10),'max_depth': range(1,20,5)}
rfc_random = RandomizedSearchCV(model2,parameters,n_iter=15,cv=5)
rfc_random.fit(X_train, y_train)
grid_parm_srf=rfc_random.best_params_
print(grid_parm_srf)

rfc = RandomForestClassifier(**grid_parm_srf)
    
model2 = rfc.fit(S_Train, y_res)
y_pred = rfc.predict(S_Test)
stacked_predict2=pd.DataFrame(y_pred,columns=['QuoteConversion_Flag'])
pd.concat([test_data["QuoteNumber"],stacked_predict2],axis=1).to_csv("/gdrive/My Drive/CIS 508/Assignment 3/stackedRF_result.csv", index = None)

RandomizedSearchCV-Random Forest
{'n_estimators': 50, 'min_samples_split': 90, 'max_depth': 16}
